In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Liver_Cancer"
cohort = "GSE212047"

# Input paths
in_trait_dir = "../../input/GEO/Liver_Cancer"
in_cohort_dir = "../../input/GEO/Liver_Cancer/GSE212047"

# Output paths
out_data_file = "../../output/preprocess/Liver_Cancer/GSE212047.csv"
out_gene_data_file = "../../output/preprocess/Liver_Cancer/gene_data/GSE212047.csv"
out_clinical_data_file = "../../output/preprocess/Liver_Cancer/clinical_data/GSE212047.csv"
json_path = "../../output/preprocess/Liver_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# Let's analyze the dataset
# The sample characteristics dictionary doesn't represent a proper dataframe
# We need to use the original data correctly and avoid manual construction
sample_characteristics = {
    0: ['strain: Lhx2 floxed;  C57BL/6J background', 'strain: Mx1Cre+; Lhx2 floxed;  C57BL/6J background'],
    1: ['treatment: 2 weeks after poly:IC induce Mx1Cre activation'],
    2: ['cell type: FACS-sorted VitA+ Hepatic Stellate Cells']
}

# 1. Gene Expression Data Availability
# The background info mentions bulk RNAseq and microarray data, which would contain gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Liver Cancer):
# Looking at the data, there's no explicit mention of cancer/non-cancer status in sample characteristics.
# This appears to be mouse liver tissue samples with genetic modifications, not human cancer samples.
trait_row = None  # Not available in the sample characteristics

# For age:
# No age data is provided in the sample characteristics
age_row = None

# For gender:
# No gender data is provided in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion

# Define conversion functions even though we won't use them
def convert_trait(value):
    if value is None:
        return None
    
    value = value.lower() if isinstance(value, str) else str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'cancer' in value or 'tumor' in value or 'hcc' in value:
        return 1
    elif 'normal' in value or 'control' in value or 'non-tumor' in value:
        return 0
    else:
        return None

def convert_age(value):
    if value is None:
        return None
    
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    if value is None:
        return None
    
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    else:
        return None

# 3. Save Metadata
# Since trait_row is None, is_trait_available is False
is_trait_available = trait_row is not None

validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this step


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 4: Gene Identifier Review

In [ ]:
# These don't appear to be standard human gene symbols, which would typically be like BRCA1, TP53, etc.
# They look like probe identifiers or some other form of ID on a microarray platform (GPL6246).
# These numeric IDs need to be mapped to actual gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify the columns for mapping
# Looking at the gene annotation preview, we need:
# - 'ID' column in gene_annotation for the probe identifiers
# - 'gene_assignment' column for the gene symbols

# Print information to help verify the mapping
print("\nChecking gene identifiers in both datasets:")
print(f"Sample IDs in gene_data: {gene_data.index[:5].tolist()}")
print(f"Sample IDs in gene_annotation: {gene_annotation['ID'][:5].tolist()}")

# 2. Get gene mapping dataframe
# First, let's examine the format of gene_assignment more closely
print("\nExamining 'gene_assignment' format for mapping strategy:")
print(gene_annotation['gene_assignment'].iloc[0])

# Create the gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')
print(f"\nGene mapping dataframe created with shape: {mapping_df.shape}")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level to gene expression
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"\nMapped gene expression data shape: {gene_data.shape}")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Mapped gene data saved to: {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the index
print("\nNormalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print("First 10 gene identifiers after normalization:")
print(normalized_gene_data.index[:10].tolist())

# Save the normalized gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Based on Step 2, we determined trait_row was None, meaning no trait data is available
print("\nNo human trait data is available in this dataset.")
is_gene_available = len(normalized_gene_data) > 0
is_trait_available = False  # We know this from Step 2
is_biased = None  # Cannot determine bias without trait data

# Create empty DataFrame for consistency in validation
linked_data_clean = pd.DataFrame()

# 5. Conduct final quality validation and save metadata
print("\nConducting final quality validation...")
note = "This dataset contains mouse gene expression data without human trait information. The data appears to be from mouse hepatic stellate cells with genetic modifications (Lhx2 floxed), not suitable for human liver cancer association studies."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

print("Dataset deemed not usable for human trait associative studies due to lack of trait data.")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the index
print("\nNormalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print("First 10 gene identifiers after normalization:")
print(normalized_gene_data.index[:10].tolist())

# Save the normalized gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Based on Step 2, we determined that trait data is not available in this dataset
print("\nNo clinical trait data is available in this dataset.")

# Create empty DataFrame for validation
# Since we determined in Step 2 that trait data is not available
empty_clinical_df = pd.DataFrame()
linked_data = pd.DataFrame()

# Set up variables for validation
is_gene_available = len(normalized_gene_data) > 0
is_trait_available = False  # We know this from Step 2
is_biased = False  # Setting a default value for validation

# 5. Conduct final quality validation and save metadata
print("\nConducting final quality validation...")
note = "This dataset contains mouse gene expression data without human trait information. The data appears to be from mouse hepatic stellate cells with genetic modifications (Lhx2 floxed), not suitable for human liver cancer association studies."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=empty_clinical_df,
    note=note
)

print("Dataset deemed not usable for human trait associative studies due to lack of trait data.")